In [1]:
# Extensions
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

# Imports
import torch
import pandas as pd
import numpy as np

# Configuration
ID = torch.Tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
x = torch.randn(10, 4, 100)
x_rev_comp = torch.randn(10, 4, 100)
y = torch.randint(0, 2, (10, 1))
batch = (ID, x, x_rev_comp, y)

# Layers

In [105]:
from eugene.models.base import _layers as layers

## Convolutional

In [127]:
layer = layers.BiConv1D(4, 8, 10)

In [128]:
test = torch.nn.Conv1d(4, 8, 10)

In [129]:
test.weight.shape

torch.Size([8, 4, 10])

In [130]:
layer.dropout_rate

In [134]:
layer(x).shape

torch.Size([10, 8, 100])

## Pooling

## Recurrent

## Normalization

## Wrappers

## Gluers

## Miscallaneous

# Activations

# Blocks

In [25]:
from eugene.models.base import _blocks as blocks

## DenseBlock

In [43]:
dense_block = blocks.DenseBlock(
    input_dim=400, 
    output_dim=1, 
    hidden_dims=[200, 100, 50, 10],
    activations="exponential",
    batchnorm=True,
    batchnorm_first=True, 
    dropout_rates=[0.1, None],
    biases=False
)
dense_block

DenseBlock(
  (layers): Sequential(
    (0): Linear(in_features=400, out_features=200, bias=False)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Exponential()
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=False)
    (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Exponential()
    (7): Linear(in_features=100, out_features=50, bias=False)
    (8): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Exponential()
    (10): Linear(in_features=50, out_features=10, bias=False)
    (11): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Exponential()
    (13): Linear(in_features=10, out_features=1, bias=False)
  )
)

In [27]:
dense_block(x.reshape(10, -1))

tensor([[-0.2320],
        [-3.0059],
        [ 0.3035],
        [-1.2020],
        [-0.5757],
        [-1.6963],
        [-0.5692],
        [-1.1548],
        [-0.6869],
        [-1.8460]], grad_fn=<MmBackward0>)

## Conv1DBlock

In [44]:
conv1d_block = blocks.Conv1DBlock(
    input_len=100,
    input_channels=4,
    conv_channels=[10, 10],
    conv_kernels=[3, 3],
    conv_strides=[1, 1],
    conv_dilations=[1, 1],
    conv_padding=["valid", "valid"],
    activations=[None, "relu"],
    pool_types="avg",
    pool_kernels=[1, 1],
    pool_strides=[1, 1],
    pool_padding=[0, 0],
    dropout_rates=[None, 0.5],
    batchnorm=True,
    batchnorm_first=True
)
conv1d_block

Conv1DBlock(
  (layers): Sequential(
    (0): Conv1d(4, 10, kernel_size=(3,), stride=(1,), padding=valid)
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
    (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
    (4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Dropout(p=0.5, inplace=False)
    (7): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  )
)

In [155]:
conv1d_block(x).shape

torch.Size([10, 10, 96])

## ReccurentBlock

In [156]:
recurrent_block = blocks.RecurrentBlock(
    input_dim=4,
    hidden_dim=10,
    num_layers=2,
    unit_type="lstm",
    bidirectional=True,
    dropout_rates=0.1,
    bias=False,
    batch_first=True
)
recurrent_block

RecurrentBlock(
  (layers): LSTM(4, 10, num_layers=2, bias=False, batch_first=True, dropout=0.1, bidirectional=True)
)

In [157]:
recurrent_block(x.transpose(1, 2))

(tensor([[[ 0.0033,  0.0251,  0.0107,  ...,  0.0047,  0.0192, -0.0047],
          [ 0.0124,  0.0193,  0.0168,  ..., -0.0035,  0.0193, -0.0111],
          [-0.0028, -0.0178, -0.0229,  ...,  0.0075,  0.0230, -0.0116],
          ...,
          [ 0.0423, -0.0010, -0.0038,  ...,  0.0162,  0.0342,  0.0061],
          [ 0.0480,  0.0186, -0.0012,  ..., -0.0028,  0.0102, -0.0073],
          [ 0.0267,  0.0014, -0.0179,  ..., -0.0097, -0.0012, -0.0096]],
 
         [[-0.0066, -0.0552, -0.0469,  ..., -0.0651, -0.0444, -0.0488],
          [ 0.0064, -0.0506, -0.0255,  ..., -0.0449, -0.0297, -0.0161],
          [-0.0320, -0.0564, -0.0312,  ..., -0.0377, -0.0394, -0.0138],
          ...,
          [ 0.0076, -0.0247, -0.0067,  ..., -0.0295, -0.0175, -0.0006],
          [-0.0448, -0.0262, -0.0207,  ..., -0.0218, -0.0055,  0.0073],
          [-0.0301, -0.0140, -0.0108,  ...,  0.0013,  0.0180,  0.0080]],
 
         [[ 0.0128,  0.0013,  0.0094,  ..., -0.0064,  0.0123,  0.0010],
          [ 0.0075, -0.0123,

## Transformer Block

In [69]:
transformer_block = blocks.TransformerBlock(
    input_dim=4,
    head_dim=10,
    num_heads=2,
)
transformer_block

In [75]:
transformer_block(x.transpose(1,2), mask=None).shape

torch.Size([10, 100, 4])

# Towers

In [43]:
from eugene.models.base import _towers as towers

In [56]:
# example
from inspect import signature
class MyBlock(torch.nn.Module):
    def __init__(self, arg1: float, arg2: str, arg3: int = 42, arg4: str = 'default'):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2
        self.arg3 = arg3
        self.arg4 = arg4
    
    def __repr__(self):
        sig_vals = ", ".join([f'{arg}={getattr(self, arg)}' for arg in signature(self.__init__).parameters.keys()])
        return f'MyBlock({sig_vals})'

In [57]:
my_tower = towers.Tower(
    block=MyBlock,
    repeats=3,
    dynamic_block_args={'arg1': 2, 'arg2': ['a', 'b', 'c']},
    mults={'arg1': 1.1, 'arg3': 1.1}
)

In [58]:
my_tower

Tower(
  (tower): Sequential(
    (0): MyBlock(arg1=2.0, arg2=a, arg3=42, arg4=default)
    (1): MyBlock(arg1=2.200000047683716, arg2=b, arg3=46, arg4=default)
    (2): MyBlock(arg1=2.4200000762939453, arg2=c, arg3=50, arg4=default)
  )
)

# Base Model

In [ ]:
from eugene.models.base._base_models import SequenceModel

# Optimizers

# Models

## Sequence to Task

### FCN

In [3]:
from eugene.models.base._sequence_models import FCN 

In [4]:
model = FCN(
    input_len=100,
    output_dim=10,
    task="regression",
    optimizer="adam",
    dense_kwargs={
        "hidden_dims": [50],
        "batchnorm": True,
    }
)
model

FCN(
  (metric): R2Score()
  (dense_block): DenseBlock(
    (layers): Sequential(
      (0): Linear(in_features=400, out_features=50, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Linear(in_features=50, out_features=10, bias=True)
    )
  )
)

In [ ]:
mode

In [26]:
for child in model.dense_block.layers.children():
    print(child, "weight" in child._parameters)

Linear(in_features=400, out_features=50, bias=True) True
ReLU() False
BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) True
Linear(in_features=50, out_features=10, bias=True) True


In [20]:
model.optimizer.__name__

'Adam'

In [21]:
model.summary()

Model: FCN
Sequence length: 100
Output dimension: 10
Strand: ss
Task: regression
Aggregation of strands: None
Loss function: mse_loss
Optimizer: Adam
	Optimizer parameters: {}
	 Optimizer learning rate: 0.001
Scheduler: ReduceLROnPlateau
	Scheduler parameters: {}
Metric: r2
	Metric parameters: {}
Seed: None
Parameters summary:


  | Name        | Type       | Params
-------------------------------------------
0 | metric      | R2Score    | 0     
1 | dense_block | DenseBlock | 20.7 K
-------------------------------------------
20.7 K    Trainable params
0         Non-trainable params
20.7 K    Total params
0.083     Total estimated model params size (MB)

('ss', 'regression', None)

In [ ]:
model.loss_fxn

<function torch.nn.functional.mse_loss(input: torch.Tensor, target: torch.Tensor, size_average: Union[bool, NoneType] = None, reduce: Union[bool, NoneType] = None, reduction: str = 'mean') -> torch.Tensor>

In [ ]:
model.optimizer, model.optimizer_lr, model.optimizer_kwargs 

(torch.optim.adam.Adam, 0.001, {})

In [ ]:
model.scheduler, model.scheduler_kwargs

(torch.optim.lr_scheduler.ReduceLROnPlateau, {})

In [ ]:
model.metric_name, model.metric

('r2', R2Score())

In [ ]:
outs = model(x).squeeze(dim=1)
outs.size()

torch.Size([10, 10])

In [ ]:
y = torch.randint(0, 2, (10, 10))
y = torch.normal(0, 1, (10, 10))
y.size()

torch.Size([10, 10])

In [ ]:
model.loss_fxn(outs, y)

tensor(1.8830, grad_fn=<MseLossBackward0>)

In [ ]:
model.calculate_metric(outs, y)

tensor(-0.5084, grad_fn=<SqueezeBackward0>)

### CNN

In [5]:
from eugene.models.base._sequence_models import CNN

In [6]:
model = CNN(
    input_len=100,
    output_dim=10,
    strand="ds",
    aggr="max",
    task="regression",
    optimizer="adam",
    conv_kwargs={
        "conv_channels": [10, 10],
        "conv_kernels": [5, 3],
        "activations": "relu",
        "pool_types": "max"
    }
)
model

CNN(
  (metric): R2Score()
  (conv1d_block): Conv1DBlock(
    (layers): Sequential(
      (0): Conv1d(4, 10, kernel_size=(5,), stride=(1,), padding=valid)
      (1): ReLU()
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
      (4): ReLU()
      (5): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (dense_block): DenseBlock(
    (layers): Sequential(
      (0): Linear(in_features=940, out_features=10, bias=True)
    )
  )
)

In [ ]:
model(x, x_rev_comp).shape

torch.Size([10, 10])

### RNN

In [7]:
from eugene.models.base._sequence_models import RNN

In [8]:
model = RNN(
    input_len=100,
    output_dim=10,
    strand="ds",
    aggr="max",
    task="regression",
    optimizer="adam",
    recurrent_kwargs={
        "hidden_dim": 10,
        "num_layers": 2,
        "bidirectional": True
    }
)
model

RNN(
  (metric): R2Score()
  (recurrent_block): RecurrentBlock(
    (layers): LSTM(4, 10, num_layers=2, batch_first=True, bidirectional=True)
  )
  (dense_block): DenseBlock(
    (layers): Sequential(
      (0): Linear(in_features=20, out_features=10, bias=True)
    )
  )
)

In [ ]:
model(x.transpose(1, 2), x_rev_comp.transpose(1, 2)).shape

torch.Size([10, 10])

### Hybrid

In [9]:
from eugene.models.base._sequence_models import Hybrid

In [ ]:
model = Hybrid(
    input_len=100,
    output_dim=10,
    strand="ts",
    aggr="max",
    task="regression",
    optimizer="adam",
    conv_kwargs={
        "conv_channels": [10, 10],
        "conv_kernels": [5, 3],
        "activations": "relu",
        "pool_types": "max"
    },
    recurrent_kwargs={
        "hidden_dim": 10,
        "num_layers": 2,
        "bidirectional": True
    }
)
model

Hybrid(
  (metric): R2Score()
  (conv1d_block): Conv1DBlock(
    (layers): Sequential(
      (0): Conv1d(4, 10, kernel_size=(5,), stride=(1,), padding=valid)
      (1): ReLU()
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
      (4): ReLU()
      (5): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (reverse_conv1d_block): Conv1DBlock(
    (layers): Sequential(
      (0): Conv1d(4, 10, kernel_size=(5,), stride=(1,), padding=valid)
      (1): ReLU()
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
      (4): ReLU()
      (5): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (recurrent_block): RecurrentBlock(
    (layers): LSTM(10, 10, num_layers=2, batch_first=True, bidirectional=True)
  )
  (rev

In [ ]:
model(x, x_rev_comp).shape

torch.Size([10, 10])

### TutorialCNN

In [10]:
from eugene.models._supervised import TutorialCNN

In [15]:
model = TutorialCNN(
    input_len=100,
    output_dim=10
)
model

TutorialCNN(
  (metric): R2Score()
  (conv1): Conv1d(4, 30, kernel_size=(21,), stride=(1,))
  (dense): Linear(in_features=30, out_features=10, bias=True)
)

In [17]:
model(x).shape

torch.Size([10, 10])

### Jores21CNN

In [18]:
from eugene.models._supervised import Jores21CNN

In [22]:
model = Jores21CNN(
    input_len=100,
    output_dim=10
)
model

Jores21CNN(
  (metric): R2Score()
  (biconv): BiConv1DBlock(
    (kernels): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 128x4x13]
        (1): Parameter containing: [torch.FloatTensor of size 128x128x13]
    )
    (biases): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 128]
        (1): Parameter containing: [torch.FloatTensor of size 128]
    )
  )
  (conv): Conv1d(128, 128, kernel_size=(13,), stride=(1,), padding=same)
  (dropout): Dropout(p=0.15, inplace=False)
  (fc): Linear(in_features=12800, out_features=64, bias=True)
  (batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [24]:
model(x).shape

torch.Size([10, 10])

In [25]:
eu.models.init_weights(model, initializer="xavier_uniform")

NameError: name 'eu' is not defined

### Kopp21CNN

In [27]:
from eugene.models._supervised import Kopp21CNN

In [1]:
from eugene.models.base._blocks import Conv1DBlock

Global seed set to 13
2022-12-11 15:39:26.205783: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Fontconfig warning: ignoring UTF-8: not a valid region tag


In [2]:
block = Conv1DBlock(
    input_len=100,
    input_channels=4,
    conv_channels=[10, 10],
    conv_kernels=[5, 3],
    conv_strides=[1, 1],
    conv_dilations=[1, 1],
    conv_padding=["valid", "valid"],
    activations=None,
    pool_types=["max", None],
    batchnorm=True,
    batchnorm_first=False
)

['max', None] [<class 'torch.nn.modules.pooling.MaxPool1d'>, None]


In [3]:
block

Conv1DBlock(
  (layers): Sequential(
    (0): Conv1d(4, 10, kernel_size=(5,), stride=(1,), padding=valid)
    (1): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    (2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
    (4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [135]:
CNN(
    input_len=100,
    output_dim=1,
    strand="ds",
    task="binary_classification",
    aggr="max",
    loss="bce",

)

NameError: name 'CNN' is not defined

In [28]:
model = Kopp21CNN(
    input_len=100,
    output_dim=10
)

/Users/adamklie/miniconda3/envs/eugene/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [30]:
model(x, x_rev_comp).shape

torch.Size([10, 10])

### FactorizedBasset

In [64]:
from eugene.models._model_zoo import FactorizedBasset

In [65]:
model = FactorizedBasset(input_len=100, output_dim=1)
model

FactorizedBasset(
  (metric): AUROC()
  (conv1d_block1): Conv1DBlock(
    (layers): Sequential(
      (0): Conv1d(4, 48, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv1d(48, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Conv1d(64, 100, kernel_size=(3,), stride=(1,), padding=(1,))
      (7): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
      (9): Conv1d(100, 150, kernel_size=(7,), stride=(1,), padding=(3,))
      (10): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (11): ReLU()
      (12): Conv1d(150, 300, kernel_size=(7,), stride=(1,), padding=(3,))
      (13): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (14):

In [66]:
model.conv1d_block3.flatten_dim

18400

In [67]:
model(x).shape

torch.Size([10, 200, 92])


torch.Size([10, 1])

# Losses

# Metrics

In [2]:
import torchmetrics
from torchmetrics import Metric

In [3]:
from torchmetrics import Metric

class MyAccuracy(Metric):
    def __init__(self):
        super().__init__()
        self.add_state("correct", default=torch.tensor(0), dist_reduce_fx="sum")
        self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        preds, target = self._input_format(preds, target)
        assert preds.shape == target.shape

        self.correct += torch.sum(preds == target)
        self.total += target.numel()

    def compute(self):
        return self.correct.float() / self.total

In [4]:
metric = MyAccuracy()

/Users/adamklie/miniconda3/envs/eugene/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (MyAccuracy). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_full_state_property`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


In [5]:
metric.__class__.__name__.lower()

'myaccuracy'

In [7]:
from eugene.models.base._sequence_models import CNN

In [38]:
model = CNN(
    input_len=100,
    output_dim=1,
    strand="ss",
    aggr="max",
    task="binary_classification",
    optimizer="adam",
    conv_kwargs={
        "conv_channels": [10, 10],
        "conv_kernels": [5, 3],
        "activations": "relu",
        "pool_types": "max"
    },
    scheduler="reduce_lr_on_plateau",
    metric="accuracy",
    seed=13
)
model

[autoreload of eugene.models failed: Traceback (most recent call last):
  File "/Users/adamklie/miniconda3/envs/eugene/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/adamklie/miniconda3/envs/eugene/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/adamklie/miniconda3/envs/eugene/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/adamklie/miniconda3/envs/eugene/lib/python3.7/importlib/__init__.py", line 160, in reload
    name=parent_name) from None
ImportError: parent 'eugene' not in sys.modules
]
Global seed set to 13


TypeError: super(type, obj): obj must be an instance or subtype of type

In [33]:
model.configure_optimizers()

{'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     maximize: False
     weight_decay: 0
 ),
 'lr_scheduler': <torch.optim.lr_scheduler.ReduceLROnPlateau at 0x7fbe3e911bd0>,
 'monitor': 'val_loss'}

In [36]:
model.lr_schedulers()

AttributeError: 'NoneType' object has no attribute 'lr_schedulers'

In [10]:
from eugene.datasets._datasets import random1000
from eugene.preprocess import ohe_seqs_sdata

In [11]:
sdata = random1000()
sdata

SeqData object with = 1000 seqs
seqs = (1000,)
names = (1000,)
rev_seqs = None
ohe_seqs = None
ohe_rev_seqs = None
seqs_annot: 'label_0', 'label_1', 'label_2', 'label_3', 'label_4', 'label_5', 'label_6', 'label_7', 'label_8', 'label_9', 'activity_0', 'activity_1', 'activity_2', 'activity_3', 'activity_4', 'activity_5', 'activity_6', 'activity_7', 'activity_8', 'activity_9'
pos_annot: None
seqsm: None
uns: None

In [12]:
ohe_seqs_sdata(sdata)

One-hot encoding sequences:   0%|          | 0/1000 [00:00<?, ?it/s]

SeqData object modified:
	ohe_seqs: None -> 1000 ohe_seqs added


In [13]:
sdataset = sdata.to_dataset(target_keys="label_0")

No transforms given, assuming just need to tensorize.


In [14]:
sdataloader = sdataset.to_dataloader(batch_size=128, shuffle=False)
ID, x, x_rev_comp, y = next(iter(sdataloader))

In [15]:
model.metric.reset()
metric_vals = []
for batch_idx, batch in enumerate(sdataloader):
    ID, x, x_rev_comp, y = batch
    outs = model(x, x_rev_comp=None).squeeze(dim=1)
    loss = model.loss_fxn(outs, y)
    print(f"Loss is {loss}", end=", ")
    met = model.metric(outs, y.long())
    print(f"{model.metric_name} is {met} at batch {batch_idx}")
    metric_vals.append(met)
    if batch_idx == 5:
        break

Loss is 0.5474207401275635, accuracy is 0.515625 at batch 0
Loss is 0.5789369344711304, accuracy is 0.4765625 at batch 1
Loss is 0.5303661823272705, accuracy is 0.5234375 at batch 2
Loss is 0.6017324924468994, accuracy is 0.453125 at batch 3
Loss is 0.5952660441398621, accuracy is 0.46875 at batch 4
Loss is 0.5732230544090271, accuracy is 0.484375 at batch 5


In [17]:
model.metric

Accuracy()

In [18]:
model.loss_fxn

<function torch.nn.functional.mse_loss(input: torch.Tensor, target: torch.Tensor, size_average: Union[bool, NoneType] = None, reduce: Union[bool, NoneType] = None, reduction: str = 'mean') -> torch.Tensor>

In [19]:
model.optimizer

torch.optim.adam.Adam

In [21]:
model.configure_optimizers()

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [20]:
model.scheduler

In [268]:
from torchmetrics.functional.classification import binary_accuracy

In [ ]:
model.metric.

In [270]:
all_preds = torch.concat(model.metric.pred)
all_targets = torch.concat(model.metric.target)
binary_accuracy(all_preds, all_targets)

AttributeError: 'Accuracy' object has no attribute 'pred'

In [271]:
model.metric.compute()

tensor(0.4870)

In [272]:
torch.mean(torch.Tensor(metric_vals))

tensor(0.4870)

In [206]:
model.calculate_metrics(probs, y)

tensor(0.5029)

tensor(0.4725)

In [28]:
batch[1].shape, batch[3].shape

(torch.Size([10, 4, 100]), torch.Size([10, 1]))

In [32]:
model._common_step(batch=batch, batch_idx=0, optimizer_idx=0, stage="train")

tensor(0.2946, grad_fn=<MseLossBackward0>)


/Users/adamklie/Desktop/EUGENe/eugene/models/base/_base_models.py:162: UserWarning: Using a target size (torch.Size([10, 1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = self.loss_fxn(outs, y)


TypeError: log() missing 2 required positional arguments: 'name' and 'value'

In [34]:
model.log_dict

<bound method LightningModule.log_dict of CNN(
  (metric): AUROC()
  (conv1d_block): Conv1DBlock(
    (layers): Sequential(
      (0): Conv1d(4, 10, kernel_size=(5,), stride=(1,), padding=valid)
      (1): ReLU()
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
      (4): ReLU()
      (5): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (dense_block): DenseBlock(
    (layers): Sequential(
      (0): Linear(in_features=940, out_features=1, bias=True)
    )
  )
)>

In [26]:
model.lr_schedulers()

AttributeError: 'NoneType' object has no attribute 'lr_schedulers'

In [29]:
model.lr_schedulers()

AttributeError: 'NoneType' object has no attribute 'lr_schedulers'